In [23]:
from models.db import get_db, SessionLocal
from models.schemas import Item, Supplier, SupplierItem, Transactions
from sqlalchemy import func, desc, case, or_
from datetime import datetime

In [7]:
db = SessionLocal()

In [39]:
item = db.query(Item.item_code, Item.item_name, Item.item_quantity, Item.updated_at).all()
item

[('0a6f4b8e', 'tf', 10, datetime.datetime(2025, 6, 22, 19, 11, 14, 398482)),
 ('b371bc46', 'u', 60, datetime.datetime(2025, 6, 22, 19, 15, 49, 112413))]

In [38]:
db.query(Transactions.item_code, Transactions.quantity, Transactions.updated_at).where(Transactions.updated_at >= datetime(2025, 6, 22, 7, 0, 0)).all()

[('0a6f4b8e', 1, datetime.datetime(2025, 6, 22, 7, 45, 27)),
 ('0a6f4b8e', 10, datetime.datetime(2025, 6, 22, 8, 15, 30)),
 ('0a6f4b8e', 18, datetime.datetime(2025, 6, 22, 8, 16, 3)),
 ('0a6f4b8e', 60, datetime.datetime(2025, 6, 22, 8, 58, 11)),
 ('0a6f4b8e', 1, datetime.datetime(2025, 6, 22, 18, 58, 47, 368790)),
 ('b371bc46', 1, datetime.datetime(2025, 6, 22, 19, 14, 57, 797735))]

In [41]:
last_updated_at = func.max(func.coalesce(Transactions.updated_at, Item.updated_at)).label("updated_at")
query = (
    db.query(
        Item.item_code,
        Item.item_name,
        (Item.item_quantity + func.sum(func.coalesce(Transactions.quantity, 0))).label("current_stock"),
        last_updated_at
    )
    .outerjoin(Transactions, Item.item_code == Transactions.item_code)
    .where(
        or_(
            Transactions.updated_at == None,
            Transactions.updated_at >= Item.updated_at
        )
    )
    .group_by(Item.item_code, Item.item_name)
    .order_by(desc(last_updated_at))
)
inventory = query.all()
inventory

[]

In [43]:
from sqlalchemy import case, func, or_, desc

quantity_case = func.sum(
    case(
        (Transactions.updated_at >= Item.updated_at, Transactions.quantity),
        else_=0
    )
)

last_updated_at = func.max(
    case(
        (Transactions.updated_at >= Item.updated_at, Transactions.updated_at),
        else_=Item.updated_at
    )
).label("updated_at")

query = (
    db.query(
        Item.item_code,
        Item.item_name,
        (Item.item_quantity + func.coalesce(quantity_case, 0)).label("current_stock"),
        last_updated_at
    )
    .outerjoin(Transactions, Item.item_code == Transactions.item_code)
    .group_by(Item.item_code, Item.item_name, Item.item_quantity, Item.updated_at)
    .order_by(desc(last_updated_at))
)

inventory = query.all()
inventory

[('b371bc46', 'u', 50, datetime.datetime(2025, 6, 22, 19, 21, 53, 966172)),
 ('0a6f4b8e', 'tf', 10, datetime.datetime(2025, 6, 22, 19, 11, 14, 398482))]